<a href="https://colab.research.google.com/github/OmerRosen/Kaggle/blob/main/IMDB_2022_Hit_or_Flop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB 2022 - Hit or Flop

## Mission Statement

### Data To Collect

#### Search box page:
* Movie Id
* Movie Title
* Movie year
* Rating
* MetaScore - Outcome
* Description
* Poster
* Directore name + Link
* Stars
* Votes - Output
* Genre

#### Main Movie Page data
* Movie Length
* Rank
* Writer
* Star #1
* Star #2
* Star #3
* User eviews
* Critic reviews
* Number of photos posted
* Storyline - Text
* Tag line
* Release Date - Month
* Country of origin
* Additional Lanaguages
* Country of filming
* Production companies
* Budget
* Opening Weekend Date
* Gross US & Canada - Output
* Opening weekend US & Canada - Output
* Gross worldwide - Output
* Color - Color
* Color - Black&White
* Sound mix - Dolby Digital
* Sound mix - Dolby Atmos
* Aspect ratio

#### Director Page:
* Director Age
* Is Top 500?
* Director Gender (Based on bio)
* Previous film count - As Director
* Previous film count - As Writer
* Previous film count - As As Producer
* Director Publicity listing count

#### Information Abount Cast:
* Full list of cast and their profile links
* Number of cast members
* Produced by - Is top 500?
* Music by - Is top 500?
* Cinematography by  - Is top 500
* Film Editing by   - Is top 500
* Art Direction by  - Is top 500
* Number of Production Management
* Number of Art Department
* Number of Sound Department
* Number of Camera and Electrical Department
* Number of Editorial Department
* Number of Music Department
* Number of Additional Crew

#### Actor Page
* Is Top 5000?
* Is Top 500?
* Is Top 100 (aka - Has numberical rank)
* Is Top 10 (aka - Has numberical rank)
* Numerical Rank (Could be none)
* Gender
* Age
* Oscar nominations 
* Birth country

#### Aggragated data:
* How many male stars
* How many female stars
* Avrage cast memeber age
* Max/Min age of cast member
* Number of unique birth countries of actors
* Total number of Oscar numinatior for cast
* Num cast members in top 5000
* Num cast memebers in top 500
* Num cast members in top 100
* Num cast members in top 10

# IMDB Scraper

## Import Libraries

In [1]:
import requests
from IPython.core.display import HTML
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import traceback


## Set main variables

In [2]:
base_url = "https://www.imdb.com"
base_folder_path = "/content/drive/My Drive/Harvard HW/Course 4 - Final Project"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 10 # Minimum vote amount to coolect movie
start_point = 1 # Start from movie #1-50

movie_search_url = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&{start_point}anguages=en&start=1&ref_=adv_nxt"
movie_search_url

'https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&1anguages=en&start=1&ref_=adv_nxt'

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

list_of_50_movies = requests.get(f'{movie_search_url}', headers=headers, timeout=10)
list_of_50_movies

<Response [200]>

In [4]:
list_of_50_movies_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser').find_all('div',{'class':'lister-item mode-advanced'})


In [5]:
movie_search_page = list_of_50_movies_soup[0]

## Function: Get Movie Box details

In [6]:
def get_basic_details(movie_search_page):
  search_box_info = {}
  search_box_info['movie_name'] = movie_search_page.find('a')
  search_box_info['movie_page_url'] = f"{base_url}{movie_search_page.find('a')['href']}"

  try:
    search_box_info['movie_place'] = None if not movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}) else movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}).text.replace('.','')
    search_box_info['movie_id'] = None if not search_box_info['movie_page_url'] else search_box_info['movie_page_url'].split('/')[-2]
    search_box_info['movie_name'] = None if not movie_search_page.find('a').find('img') else movie_search_page.find('a').find('img')['alt'].strip()
    search_box_info['movie_thubmnail'] = None if not movie_search_page.find('a').find('img') else movie_search_page.find('a').find('img')['src']
    search_box_info['movie_metascore'] = None if not movie_search_page.find('span', {'class': 'metascore'}) else movie_search_page.find('span', {'class': 'metascore'}).text.strip()
    search_box_info['movie_description'] = None if not movie_search_page.find_all('p',{'class':'text-muted'}) else movie_search_page.find_all('p',{'class':'text-muted'})[1].text.strip()
    search_box_info['runtime_min'] = None if not movie_search_page.find('span',{'class':'runtime'}) else movie_search_page.find('span',{'class':'runtime'}).text.split(' ')[0]

    bottom_box_info = movie_search_page.find('p',{'class':'sort-num_votes-visible'}).find_all('span')
    search_box_info['movie_vote_num'] = bottom_box_info[1]['data-value']

    search_box_info['movie_rating'] = None if not movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}) else movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}).text.strip()
    search_box_info['movie_genere'] = None if not movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}) else movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}).text.strip()

    search_box_info['__SuccsefullyCollectBasicDetails'] = True;

  except Exception as e:
    print(f"Failed extracting data for movie: {search_box_info['movie_name']}. \nUrl: {search_box_info['movie_page_url']}.\n Error:\n{e}")
    traceback.print_exc()
    search_box_info['__SuccsefullyCollectBasicDetails'] = False;


  return search_box_info

In [7]:
movie_search_page = list_of_50_movies_soup[1]
search_box_info = get_basic_details(movie_search_page)
search_box_info



{'movie_name': 'Everything Everywhere All at Once',
 'movie_page_url': 'https://www.imdb.com/title/tt6710474/?ref_=adv_li_i',
 'movie_place': '2',
 'movie_id': 'tt6710474',
 'movie_thubmnail': 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png',
 'movie_metascore': '81',
 'movie_description': 'A middle-aged Chinese immigrant is swept up into an insane adventure in which she alone can save existence by exploring other universes and connecting with the lives she could have led.',
 'runtime_min': '139',
 'movie_vote_num': '423762',
 'movie_rating': 'R',
 'movie_genere': 'Action, Adventure, Comedy',
 '__SuccsefullyCollectBasicDetails': True}

In [8]:
movie_main_page = requests.get(search_box_info['movie_page_url'], headers=headers)
movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')

## Function: Extract Movie Artists

In [9]:
def extract_artist(artist_type, movie_main_page_soup):

  item_dict = {}

  search_result = movie_main_page_soup.find(string=f"{artist_type}s") if movie_main_page_soup.find(string=artist_type) is None else movie_main_page_soup.find(string=artist_type)
  search_item_list = []
  if search_result!=None:
    search_item_list = search_result.find_parent().find_next_sibling().find_all('a')
  
  item_dict[f"{artist_type}_count"] = len(search_item_list)

  for i,item in enumerate(search_item_list):
    artisc_name = item.text.strip()
    artisc_url = item['href']
    artist_id = artisc_url.split('/')[2]
    item_dict[f"{artist_type}_{i+1}_name"] = artisc_name
    #item_dict[f"{artist_type}_{i+1}_url"] = artisc_url
    item_dict[f"{artist_type}_{i+1}_imdb_id"] = artist_id

  return item_dict

print(extract_artist("Director", movie_main_page_soup))
print(extract_artist("Writer", movie_main_page_soup))
print(extract_artist("Star", movie_main_page_soup))

{'Director_count': 2, 'Director_1_name': 'Daniel Kwan', 'Director_1_imdb_id': 'nm3453283', 'Director_2_name': 'Daniel Scheinert', 'Director_2_imdb_id': 'nm3215397'}
{'Writer_count': 2, 'Writer_1_name': 'Daniel Kwan', 'Writer_1_imdb_id': 'nm3453283', 'Writer_2_name': 'Daniel Scheinert', 'Writer_2_imdb_id': 'nm3215397'}
{'Star_count': 3, 'Star_1_name': 'Michelle Yeoh', 'Star_1_imdb_id': 'nm0000706', 'Star_2_name': 'Stephanie Hsu', 'Star_2_imdb_id': 'nm3513533', 'Star_3_name': 'Jamie Lee Curtis', 'Star_3_imdb_id': 'nm0000130'}


## Function: Perform Currency Conversion

In [10]:
currency_codes = {
    "$":"USD",
    "€":"EUR",
    "£":"GBP",
    "₹":"INR",
    "â‚¬": "EUR",
    "â‚¹": "INR",
    "CA$": "CAD",
    "NOKÂ": "NOK",
    "Â£": "GBP",
    "CHF": "CHF",
    "Â¥": "JPY",
    "PKR": "PKR",
    "A$": "AUD",
    "CZK": "CZK",
    "RUR": "RUB",
    "NZ$": "NZD",
    "MYR": "MYR",
    "NGN": "NGN",
    "NOK":"NOK",
    "A$":"AUD",
    "â‚¬":"EUR",
    "â‚¹":"INR",
    "Â£":"GBP",
    "CA$":"CAD",
    "CHFÂ ":"CHF",
    "MYRÂ ":"MYR",
    "NGNÂ ":"NGN",
    "NOKÂ ":"NOK",
    "NZ$":"NZD",
    "PKRÂ ":"PKR",
    "RURÂ ":"RUB"
}


currency_conversion_values = {}


In [11]:
def extract_currency_symbol(s):
    match = re.search(r"[^\d]+", s)
    if match:
        return match.group(0).strip()
    else:
        return ""

In [12]:
def get_currency_code(amount_string):
  currency_symbol = extract_currency_symbol(amount_string)
  if currency_symbol in currency_codes:
      return currency_codes[currency_symbol]
  else:
    print(f"Could not find a value for: {currency_symbol} in {amount_string}")
    return None

In [13]:
def extract_numerical_value(string):
    # Remove all non-numeric characters from the string
    numerical_string = re.sub(r"[^\d.]+", "", string)
    # Convert the string to a float and return it
    return float(numerical_string)

In [14]:

with open(f'{base_folder_path}/openexchangerates.txt', 'r') as f:
    api_key = f.read().strip()

api_key

def convert_string_amount_to_usd(amount):

  # get the currency code and amount value
  currency_code = get_currency_code(amount)
  original_amount = extract_numerical_value(amount)
  usd_amount = None

  if currency_code==None:
    usd_amount=None
  elif currency_code!="USD":
  
    if currency_code in currency_conversion_values:
      exchange_rate = currency_conversion_values[currency_code]
    else:
      # make API request to get exchange rate for the currency code
      url = f"https://openexchangerates.org/api/latest.json?app_id={api_key}&symbols={currency_code}"
      print(amount,currency_code,original_amount,url)
      response = requests.get(url)
      print(response)
      # parse the exchange rate from the API response
      exchange_rate = response.json()["rates"][currency_code]
      
      currency_conversion_values[currency_code] = exchange_rate
      
      # calculate the USD equivalent amount
      usd_amount = round(original_amount / exchange_rate)
      
      print(f"{amount} {currency_code} = {usd_amount} USD")
  else:
    usd_amount = original_amount

  return usd_amount,original_amount,currency_code



convert_string_amount_to_usd("CA$15,000")
convert_string_amount_to_usd("NOKÂ 80,200,000")
convert_string_amount_to_usd("NOKÂ 20,000,000")
convert_string_amount_to_usd("â‚¹3,500,000,000")
currency_conversion_values

CA$15,000 CAD 15000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=CAD
<Response [200]>
CA$15,000 CAD = 11168 USD
NOKÂ 80,200,000 NOK 80200000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=NOK
<Response [200]>
NOKÂ 80,200,000 NOK = 7685181 USD
â‚¹3,500,000,000 INR 3500000000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=INR
<Response [200]>
â‚¹3,500,000,000 INR = 42705321 USD


{'CAD': 1.343128, 'NOK': 10.435669, 'INR': 81.957}

## Function: Get Movie Page Details

In [15]:
def get_extended_datails(movie_page_url, movie_name):
  movie_page_dict = {}

  try:

    movie_main_page = requests.get(movie_page_url, headers=headers)
    movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')   

    year_rating = movie_main_page_soup.find('ul',{"class":"ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt"}).find_all('a')

    movie_page_dict['movie_year'] = year_rating[0].text if len(year_rating)>0 else None
    movie_page_dict['movie_rating'] = year_rating[1].text if len(year_rating)>1 else None

    director_dict = extract_artist("Director", movie_main_page_soup)
    if director_dict[f"Director_count"] == 0:
      print(f"No Director found for movie {movie_name}. Url: {movie_page_url}")
    movie_page_dict.update(director_dict)

    writer_dict = extract_artist("Writer", movie_main_page_soup)
    if writer_dict[f"Writer_count"] == 0:
      print(f"No Writer found for movie {movie_name}. Url: {movie_page_url}")
    movie_page_dict.update(writer_dict)

    star_dict = extract_artist("Star", movie_main_page_soup)
    if star_dict[f"Star_count"] == 0:
      print(f"No Star found for movie {movie_name}. Url: {movie_page_url}")
    movie_page_dict.update(star_dict)

    review_scores = movie_main_page_soup.find_all('span',{'class':'score'})
    movie_page_dict['user_reviews_count'] = review_scores[0].text if len(review_scores)>0 else None
    movie_page_dict['critic_reviews_count'] = review_scores[1].text if len(review_scores)>1 else None

    movie_page_dict['release_date'] = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'}, string="Release date").find_parent().find('a',{'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text.split('(')[0].strip()
    movie_page_dict['release_date']

    movie_page_dict['budget'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Budget")
    if (movie_page_dict['budget']!=None):
      movie_page_dict['budget'] = movie_page_dict['budget'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

      usd_amount,original_amount,currency_code = convert_string_amount_to_usd(movie_page_dict['budget'])
      movie_page_dict['budget_usd'] = usd_amount
      movie_page_dict['budget_currency'] = currency_code

    movie_page_dict['gross_worldwide'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross worldwide")
    if (movie_page_dict['gross_worldwide']!=None):
      movie_page_dict['gross_worldwide'] = movie_page_dict['gross_worldwide'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()
      movie_page_dict['gross_worldwide'], _, _ = convert_string_amount_to_usd(movie_page_dict['gross_worldwide'])

    movie_page_dict['gross_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross US & Canada")
    if (movie_page_dict['gross_us_canada']!=None):
      movie_page_dict['gross_us_canada'] = movie_page_dict['gross_us_canada'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()
      movie_page_dict['gross_us_canada'], _, _ = convert_string_amount_to_usd(movie_page_dict['gross_us_canada'])

    movie_page_dict['opening_weekend_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Opening weekend US & Canada")
    if (movie_page_dict['opening_weekend_us_canada']!=None):
      movie_page_dict['opening_weekend_us_canada'] = movie_page_dict['opening_weekend_us_canada'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()
      movie_page_dict['opening_weekend_us_canada'], _, _ = convert_string_amount_to_usd(movie_page_dict['opening_weekend_us_canada'])

    movie_page_dict['origin_country'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Country of origin")
    if (movie_page_dict['origin_country']!=None):
      movie_page_dict['origin_country'] = movie_page_dict['origin_country'].find_next_sibling().find('a').text.split('(')[0].strip()

    languages = movie_main_page_soup.find('span',string="Language") if movie_main_page_soup.find('span',string="Languages") is None else movie_main_page_soup.find('span',string="Languages")
    languages = [] if not languages else languages.find_next_sibling().find_all('a')
    movie_page_dict['languages'] = ",".join([language.text for language in languages])

    movie_page_dict['__SuccsefullyCollectExtandedDetails'] = True;

    

  except Exception as e:
    print(f"Failed extracting data for movie: {movie_name}. \nUrl: {movie_page_url}.\n Error:\n{e}")
    traceback.print_exc()
    movie_page_dict['__SuccsefullyCollectExtandedDetails'] = False;

  return movie_page_dict


get_extended_datails(search_box_info['movie_page_url'],search_box_info['movie_name'])

{'movie_year': '2022',
 'movie_rating': 'R',
 'Director_count': 2,
 'Director_1_name': 'Daniel Kwan',
 'Director_1_imdb_id': 'nm3453283',
 'Director_2_name': 'Daniel Scheinert',
 'Director_2_imdb_id': 'nm3215397',
 'Writer_count': 2,
 'Writer_1_name': 'Daniel Kwan',
 'Writer_1_imdb_id': 'nm3453283',
 'Writer_2_name': 'Daniel Scheinert',
 'Writer_2_imdb_id': 'nm3215397',
 'Star_count': 3,
 'Star_1_name': 'Michelle Yeoh',
 'Star_1_imdb_id': 'nm0000706',
 'Star_2_name': 'Stephanie Hsu',
 'Star_2_imdb_id': 'nm3513533',
 'Star_3_name': 'Jamie Lee Curtis',
 'Star_3_imdb_id': 'nm0000130',
 'user_reviews_count': '3.3K',
 'critic_reviews_count': '410',
 'release_date': 'April 8, 2022',
 'budget': '$14,300,000',
 'budget_usd': 14300000.0,
 'budget_currency': 'USD',
 'gross_worldwide': 138831036.0,
 'gross_us_canada': 77147981.0,
 'opening_weekend_us_canada': 501305.0,
 'origin_country': 'United States',
 'languages': 'English,Mandarin,Cantonese',
 '__SuccsefullyCollectExtandedDetails': True}

In [16]:
# print(search_box_info['movie_page_url'],search_box_info['movie_name'])
# cast_url = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'})['href']
# cast_url = f"{base_url}{cast_url}"
# cast_url

In [17]:
# Dynamic parameters:

def get_50_movie_batch(start_point, minimum_votes = 20, start_date = "2022-01-01", end_date = "2022-12-31"):
  movie_search_url_50_batch = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&user_rating=1.0,10.0&countries=us&languages=en&start={start_point}&ref_=adv_nxt"
  #print(f"Start point: {start_point}. search url: {movie_search_url_50_batch}")
  search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
  search_page_soup = BeautifulSoup(search_page.text, 'html.parser')
  list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

  max_num_of_results = search_page_soup.find('div',{'class':'desc'}).find('span').text.split(' ')[2].replace(",", "")
  max_num_of_results = int(max_num_of_results)

  #print(f"max_num_of_results: {max_num_of_results}")
  return(list_of_50_movies_soup,max_num_of_results)

list_of_50_movies_soup,max_num_of_results = get_50_movie_batch(start_point = 1, minimum_votes = 20, start_date = "2022-01-01", end_date = "2022-12-31")

In [18]:
# Fixed parameters:
base_url = "https://www.imdb.com"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 20 # Minimum vote amount to coolect movie

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}

In [19]:
movie_dataset_path = "/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv"
if os.path.exists(movie_dataset_path):
  movie_df = pd.read_csv(movie_dataset_path)
  total_movie_dataset = movie_df.set_index('movie_id').T.to_dict()
  
else:
  total_movie_dataset = {}
  movie_df = pd.DataFrame(total_movie_dataset)

print(f"total_movie_dataset contains {len(total_movie_dataset)} records")
movie_df.head(3)

total_movie_dataset contains 1350 records


,movie_id,__SuccsefullyCollectBasicDetails,__SuccsefullyCollectExtandedDetails,movie_name,movie_page_url,movie_place,movie_id.1,movie_thubmnail,movie_metascore,movie_description,...,gross_worldwide,gross_us_canada,opening_weekend_us_canada,origin_country,languages,Director_2_name,Director_2_imdb_id,Director_3_name,Director_3_imdb_id,movie_id.2
0,tt1630029,True,True,Avatar: The Way of Water,https://www.imdb.com/title/tt1630029/?ref_=adv...,1,tt1630029,https://m.media-amazon.com/images/S/sash/4Fyxw...,67.0,Jake Sully lives with his newfound family form...,...,2.314813e+09,682744586.0,134100226.0,United States,English,NaN,NaN,NaN,NaN,NaN
1,tt6710474,True,True,Everything Everywhere All at Once,https://www.imdb.com/title/tt6710474/?ref_=adv...,2,tt6710474,https://m.media-amazon.com/images/S/sash/4Fyxw...,81.0,A middle-aged Chinese immigrant is swept up in...,...,1.388310e+08,77147981.0,501305.0,United States,"English,Mandarin,Cantonese",Daniel Scheinert,nm3215397,NaN,NaN,NaN
2,tt13833688,True,True,The Whale,https://www.imdb.com/title/tt13833688/?ref_=ad...,3,tt13833688,https://m.media-amazon.com/images/S/sash/4Fyxw...,60.0,"A reclusive, morbidly obese English teacher at...",...,5.497434e+07,17453440.0,332152.0,United States,English,NaN,NaN,NaN,NaN,NaN


## Cursor: Collecting movies from IMDB

In [ ]:
start_point = 1 # Start from movie #1-50

basic_data_collected = 0
basic_data_skipped = 0
extended_data_collected = 0
extended_data_skipped = 0

count = 1
while start_point < max_num_of_results-50:

  list_of_50_movies_soup,_ = get_50_movie_batch(start_point = start_point, minimum_votes = 20, start_date = "2022-01-01", end_date = "2022-12-31")

  for i,movie in enumerate(list_of_50_movies_soup):
    movie_data = {'__SuccsefullyCollectBasicDetails':False, '__SuccsefullyCollectExtandedDetails':False}

    basic_data = get_basic_details(movie)
    movie_id = basic_data['movie_id']

    try:
      if movie_id in total_movie_dataset.keys():
        movie_data = total_movie_dataset[movie_id]

      if movie_data['__SuccsefullyCollectBasicDetails']==True:
        basic_data_skipped += 1
      else:
        basic_data_collected += 1
        movie_data.update(basic_data)

      if movie_data['__SuccsefullyCollectExtandedDetails']==True:
        extended_data_skipped += 1
      else:
        extended_data_collected += 1
        extended_data = get_extended_datails(basic_data['movie_page_url'],basic_data['movie_name'])
        movie_data.update(extended_data)

        sleep(0.1)
    except Exception as e:
      print(f"Failed running cursor for movie: {movie_id}. \nUrl: {basic_data['movie_page_url']}.\n Error:\n{e}")
      traceback.print_exc()

    total_movie_dataset[movie_id] = movie_data
    count+= 1

    if (count%200 == 0):
      print(f"\nRun number: {count}. basic_data_collected: {basic_data_collected}. extended_data_collected: {extended_data_collected}. basic_data_skipped: {basic_data_skipped}. extended_data_skipped: {extended_data_skipped}")


  movie_dataset = pd.DataFrame(total_movie_dataset).T
  movie_dataset.to_csv(movie_dataset_path, index=True, index_label="movie_id")

  start_point += 50

print(f"\nRun number: {count}. \nbasic_data_collected: {basic_data_collected}. \nextended_data_collected: {extended_data_collected}. \nbasic_data_skipped: {basic_data_skipped}. \nextended_data_skipped: {extended_data_skipped}\n")


Run number: 200. basic_data_collected: 0. extended_data_collected: 0. basic_data_skipped: 199. extended_data_skipped: 199

Run number: 400. basic_data_collected: 0. extended_data_collected: 0. basic_data_skipped: 399. extended_data_skipped: 399
Failed extracting data for movie: Gasoline Alley. 
Url: https://www.imdb.com/title/tt14174168/?ref_=adv_li_i.
 Error:
'NoneType' object has no attribute 'find_all'


Traceback (most recent call last):
  File "<ipython-input-15-8a146623238c>", line 24, in get_extended_datails
    star_dict = extract_artist("Star", movie_main_page_soup)
  File "<ipython-input-9-5bc69bceb996>", line 8, in extract_artist
    search_item_list = search_result.find_parent().find_next_sibling().find_all('a')
AttributeError: 'NoneType' object has no attribute 'find_all'



Run number: 600. basic_data_collected: 0. extended_data_collected: 1. basic_data_skipped: 599. extended_data_skipped: 598

Run number: 800. basic_data_collected: 0. extended_data_collected: 1. basic_data_skipped: 799. extended_data_skipped: 798


In [ ]:
movie_dataset = pd.DataFrame(total_movie_dataset).T
movie_dataset.to_csv(movie_dataset_path, index=True, index_label="movie_id")

movie_dataset.head(5)

## Set actor variables

In [ ]:
import requests
from IPython.core.display import HTML
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import traceback

In [ ]:
movie_dataset = pd.read_csv('/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv')
movie_dataset.head(5)

## Function: Assume Gender

In [ ]:
def assume_gender(artist_bio):
  # Define the pronouns associated with each gender
  male_pronouns = ["he", "him", "his", "himself"]
  female_pronouns = ["she", "her", "hers", "herself"]
  nonbinary_pronouns = ["they", "them", "their", "themselves"]

  # Count the number of male and female pronouns in the bio
  prnoun_count = {'Male':0, "Female":0, "NonBinary":0}

  for word in artist_bio.lower().split():
      if word in male_pronouns:
          prnoun_count["Male"] += 1
      elif word in female_pronouns:
          prnoun_count["Female"] += 1
      elif word in nonbinary_pronouns:
          prnoun_count["NonBinary"] += 1

  # Determine the actor's gender based on the pronoun count
  sorted_d = {k: v for k, v in sorted(prnoun_count.items(), key=lambda item: item[1], reverse=True)}
  Gender = next(iter(sorted_d))

  if prnoun_count[Gender] == 0: # If no prnoune was found
    Gender = "Unknown"

  return Gender

# Extract the bio text
bio_text = "Joseph Vincent Russo is an American filmmaker and producer who works alongside his brother Anthony Russo. They have directed You, Me and Dupree, Cherry and the Marvel films Captain America: The Winter Soldier, Captain America: Civil War, Avengers: Infinity War and Avengers: Endgame. Endgame is one of the highest grossing films of all time."
assume_gender(bio_text)

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil import parser

def extract_artist_age(date_string):
  try:
    # Parse the date string into a datetime object
    date_obj = parser.parse(date_string)
    # Check if the date is in the future, if so - Reduce 100 years
    if date_obj.year > datetime.now().year:
      date_obj = date_obj.replace(year=date_obj.year - 100)
    #print(date_obj)
    # Calculate the difference between the current date and the date of birth
    age = relativedelta(datetime.now(), date_obj).years
    # Return the age as an integer
    return age
  except:
    #print(f"Cannot extract age from value: {date_string}")
    return None

print(extract_artist_age("11/25/1990"))
print(extract_artist_age("16-Apr-02"))
print(extract_artist_age(""))

## Function: Collect Rank

In [ ]:
def collect_artist_rank(artist_page_soup,artist_dict):
  artist_rank = artist_page_soup.find('span',{'class':'sc-d462a8ef-6 hOuQwM starmeter-current-rank'}).text.strip()

  artist_dict['artist_rank'] = artist_rank
  artist_dict['Is_5000'] = False
  artist_dict['Is_500'] = False
  artist_dict['Is_100'] = False
  artist_dict['Is_10'] = False

  if artist_rank.lower()!="see rank":
    if artist_rank=="Top 5,000":
      artist_dict['Is_5000'] = True
    elif artist_rank=="Top 500":
      artist_dict['Is_500'] = True
      artist_dict['Is_5000'] = True
    else:
      try:
        rank = int(artist_rank)   # Convert the string to an integer
        if rank <= 100:           # Check if the rank is within the top 100
          artist_dict['Is_100'] = True
          artist_dict['Is_500'] = True
          artist_dict['Is_5000'] = True
        if rank <= 10: 
          artist_dict['Is_10'] = True
      except ValueError:
        print(f"Could not interpret value {artist_rank} for artist: {artist_dict['artist_imdb_id']}. Url: {artist_dict['artist_url']}")

artist_imdb_id = "nm14727093"
artist_page = requests.get(f"https://www.imdb.com/name/{artist_imdb_id}/", headers=headers)
artist_page_soup = BeautifulSoup(artist_page.text, 'html.parser') 
artist_dict = {}
artist_dict['artist_imdb_id'] = artist_imdb_id
artist_dict['artist_type'] = "Director"
artist_dict['artist_url'] = artist_page
collect_artist_rank(artist_page_soup,artist_dict)

## Function: Extract artist info

In [ ]:
def extract_artist_info(artist_imdb_id,col_name):
  artist_dict = {}

  artist_url_page = f"{base_url}/name/{artist_imdb_id}"
  artist_dict['artist_imdb_id'] = artist_imdb_id
  artist_dict['artist_type'] = col_name.split('_')[0]
  artist_dict['artist_url'] = artist_url_page

  try:
    artist_page = requests.get(artist_url_page, headers=headers)
    artist_page_soup = BeautifulSoup(artist_page.text, 'html.parser') 

    artist_name = artist_page_soup.find('h1',{'data-testid':'hero__pageTitle'}).find('span').text.strip()
    artist_dict['artist_name'] = artist_name

    if artist_page_soup.find('span', string="Born") is not None:
      artist_birthday = artist_page_soup.find('span',string="Born").findNextSibling().text
      artist_dict['artist_birthday'] = artist_birthday
      artist_dict['artist_age'] = extract_artist_age(artist_birthday)
      if artist_dict['artist_age'] == None:
        print(f"Unable to extract age for {artist_name}. Value: {artist_birthday} Url: {artist_url_page}")
      
    artist_bio = artist_page_soup.find('div',{'class':'ipc-html-content-inner-div'})
    if artist_bio!=None:
      artist_gender = assume_gender(artist_bio.text)
      artist_dict['artist_gender'] = artist_gender
    else:
      artist_dict['artist_gender'] = "Unknown"

    collect_artist_rank(artist_page_soup,artist_dict)

    artist_dict['award_desc'] = ""
    awards = artist_page_soup.find('div',{'data-testid':'awards'})
    if awards!=None:
      award_desc = awards.find('a').text
      artist_dict['award_desc'] = award_desc

    artist_title = artist_dict['artist_type']
    if artist_title == "Star":
      artist_title = "Actress" if artist_gender=="Female" else "Actor"

    if artist_page_soup.find('h3',string=artist_title) is not None:
      previous_work = artist_page_soup.find('h3',string=artist_title).find_next().find('li', string="Previous").find_next_sibling().text
    elif artist_page_soup.find('li',string="Previous") is not None:
      previous_work = artist_page_soup.find('li',string="Previous").find_next_sibling().text
    else:
      previous_work = 0
      print(f"Unable to get previous work for {artist_title}: {artist_name}. Url: {artist_url_page}")
      
    artist_dict['previous_work'] = previous_work
    artist_dict['__SuccsefullyCollectArtistDetails'] = True;

    return artist_dict
  except Exception as e:
    print(f"Failed extracting data for artist: {artist_imdb_id}. Url: {artist_url_page}.\n Error:\n{e}")
    traceback.print_exc()
    artist_dict['__SuccsefullyCollectArtistDetails'] = False;

artist_imdb_id = "nm10128408"
col_name = "Director_3_imdb_id"
extract_artist_info(artist_imdb_id,col_name)

## Set Artist variables

In [ ]:
artists_dataset_path = "/content/drive/My Drive/Harvard HW/Course 4 - Final Project/artists_datasets.csv"
if os.path.exists(artists_dataset_path):
  artists_df = pd.read_csv(artists_dataset_path)
  artists_list = artists_df.set_index('artist_imdb_id').T.to_dict()
  
else:
  artists_list = {}
  artists_df = pd.DataFrame(artists_list)

print(f"artists_list contains {len(artists_list)} records")
artists_df.head(3)

## Cursor: Collect Artists

In [ ]:
artist_cols = [col for col in movie_dataset.columns if "imdb_id" in col]

artists_skipped = 0
artists_collected = 0
artist_count = 1
for i, row in movie_dataset.iterrows():
  artist_dict = {'artist_collected':False}
  for col_name in artist_cols:
    artist_imdb_id = row[col_name]
    if type(artist_imdb_id) != float:
      if artist_imdb_id in artists_list.keys() and artists_list[artist_imdb_id]['artist_collected']==True:
        artist_dict = artists_list[artist_imdb_id]
        artist_dict['artist_collected']=True
        artists_skipped += 1
      else:
        artist_output = extract_artist_info(artist_imdb_id,col_name)
        if artist_output!=None:
          artist_dict.update(artist_output)
          artist_dict['artist_collected']=True
        artists_collected += 1 
        sleep(0.1)

      artists_list[artist_imdb_id] = artist_dict
      artist_count += 1
      if artist_count%200 == 0:
        artist_dataset = pd.DataFrame(artists_list).T
        artist_dataset.to_csv(artists_dataset_path, index=True, index_label="artist_imdb_id")
        print(f"\nSaving progress. Run {artist_count} for movie {i}. artists_collected:{artists_collected}. artists_skipped:{artists_skipped}")

artist_dataset = pd.DataFrame(artists_list).T
artist_dataset.to_csv(artists_dataset_path, index=True, index_label="artist_imdb_id")
print(f"\n\nFinished process. Total: {artist_count}. artists_collected:{artists_collected}. artists_skipped:{artists_skipped}")

In [ ]:
artist_dataset.head(5)